In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import time
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('Airline_Delay_Cause.csv')
df.info()
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Airline_Delay_Cause.csv'

In [ ]:
# Drop rows with missing key columns needed for target/label creation
df = df.dropna(subset=["arr_delay", "arr_flights", "arr_del15"]).reset_index(drop=True)

# Remove rows where arr_flights is zero to avoid division by zero
df = df[df["arr_flights"] > 0].reset_index(drop=True)

print("Dataset shape after cleaning:", df.shape)
print(df.head())

NameError: name 'df' is not defined

In [5]:
# One-hot encode categorical and object features
cat_cols = df.select_dtypes(include=["object", "category"]).columns
print("\nCategorical columns:", list(cat_cols))

df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
df_encoded.head()


Categorical columns: ['carrier', 'carrier_name', 'airport']


,year,month,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,...,airport_VLD,airport_VPS,airport_WRG,airport_WYS,airport_XNA,airport_XWA,airport_YAK,airport_YKM,airport_YNG,airport_YUM
0,2023,8,89.0,13.0,2.25,1.60,3.16,0.0,5.99,2.0,...,False,False,False,False,False,False,False,False,False,False
1,2023,8,62.0,10.0,1.97,0.04,0.57,0.0,7.42,0.0,...,False,False,False,False,False,False,False,False,False,False
2,2023,8,62.0,10.0,2.73,1.18,1.80,0.0,4.28,1.0,...,False,False,False,False,False,False,False,False,False,False
3,2023,8,66.0,12.0,3.69,2.27,4.47,0.0,1.57,1.0,...,False,False,False,False,False,False,False,False,False,False
4,2023,8,92.0,22.0,7.76,0.00,2.96,0.0,11.28,2.0,...,False,False,False,False,False,False,False,False,False,False


In [6]:
# Create target variables 
X = df_encoded.drop("arr_delay", axis=1)
y = df_encoded["arr_delay"]

# Classification target: high vs low delay_rate
delay_rate_threshold = 0.25   # 25% of flights delayed >= 15 minutes
df_encoded["High_Delay"] = (df["delay_rate"] > delay_rate_threshold).astype(int)

In [7]:
# Split the data into train, validation, and test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=(1/3), random_state=42
)

In [8]:
# Normalize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Logistic Regression
start = time.time()
logreg = LogisticRegression(random_state=42, max_iter=1000)
logreg.fit(X_train_scaled, y_train)
log_time = (time.time() - start) * 1000  # ms

: 